# RAG-based Question Answering

In [1]:
import ollama
from qdrant_client import QdrantClient, models
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv


In [2]:
pdf_path = "A_Review_on_Large_Language_Models_Architectures_Applications_Taxonomies_Open_Issues_and_Challenges.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [5]:
documents[0]

Document(metadata={'source': 'A_Review_on_Large_Language_Models_Architectures_Applications_Taxonomies_Open_Issues_and_Challenges.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/378289524\nA Review on Large Language Models: Architectures, Applications,\nTaxonomies, Open Issues and Challenges\nArticle\xa0\xa0in \xa0\xa0IEEE Access · January 2024\nDOI: 10.1109/ACCESS.2024.3365742\nCITATIONS\n154\nREADS\n2,578\n9 authors, including:\nMohaimenul Azam Khan Raiaan\nCharles Darwin University\n11 PUBLICATIONS\xa0\xa0\xa0298 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nSaddam Mukta\nLappeenranta – Lahti University of Technology LUT\n90 PUBLICATIONS\xa0\xa0\xa0848 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nKaniz Fatema\n20 PUBLICATIONS\xa0\xa0\xa0288 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nNur Mohammad Fahad\nUnited International University\n13 PUBLICATIONS\xa0\xa0\xa0233 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content foll

### Recursive Character Text Spitter

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    separators=["\n\n", "\n", "."]
)

In [4]:
chunks = text_splitter.split_documents(documents)

In [5]:
len(chunks)

201

In [3]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\katar\AppData\Local\Temp\ipykernel_10768\1810415143.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [4]:
load_dotenv(override=True)

url = os.getenv('QC_URL')
api_key = os.getenv('QC_API_KEY')

qdrant_client = QdrantClient(
    url=url, 
    api_key=api_key,
)

In [5]:
RECURSIVE_COLLECTION_NAME = "RecursiveSptitterEmbeddings"
if not qdrant_client.collection_exists(RECURSIVE_COLLECTION_NAME):
    qdrant_client.create_collection(
        collection_name=RECURSIVE_COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=384, distance=models.Distance.COSINE
        ),
    )

In [57]:
for index, chunk in enumerate(chunks):
    text = chunk.page_content
    embedding = embeddings_model.embed_query(text)
    qdrant_client.upsert(
        collection_name=RECURSIVE_COLLECTION_NAME,
        points=[{"id": index, "vector": embedding, "payload": {"text": text}}]
    )

### Semantic Chunker

In [6]:
texts = [doc.page_content for doc in documents]
semantic_chunker = SemanticChunker(embeddings_model)
semantic_chunks = semantic_chunker.create_documents(texts)

In [10]:
len(semantic_chunks)

192

In [12]:
semantic_chunks[0]

Document(metadata={}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/378289524\nA Review on Large Language Models: Architectures, Applications,\nTaxonomies, Open Issues and Challenges\nArticle\xa0\xa0in \xa0\xa0IEEE Access · January 2024\nDOI: 10.1109/ACCESS.2024.3365742\nCITATIONS\n154\nREADS\n2,578\n9 authors, including:\nMohaimenul Azam Khan Raiaan\nCharles Darwin University\n11 PUBLICATIONS\xa0\xa0\xa0298 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nSaddam Mukta\nLappeenranta – Lahti University of Technology LUT\n90 PUBLICATIONS\xa0\xa0\xa0848 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nKaniz Fatema\n20 PUBLICATIONS\xa0\xa0\xa0288 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nNur Mohammad Fahad\nUnited International University\n13 PUBLICATIONS\xa0\xa0\xa0233 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Mohaimenul Azam Khan Raiaan on 24 February 2024. The user has requested enhancement of the do

In [6]:
SEMANTIC_COLLECTION_NAME = "SemanticSptitterEmbeddings"
if not qdrant_client.collection_exists(SEMANTIC_COLLECTION_NAME):
    qdrant_client.create_collection(
        collection_name=SEMANTIC_COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=384, distance=models.Distance.COSINE
        ),
    )

In [13]:
for index, chunk in enumerate(semantic_chunks):
    text = chunk.page_content
    embedding = embeddings_model.embed_query(text)
    qdrant_client.upsert(
        collection_name=SEMANTIC_COLLECTION_NAME,
        points=[{"id": index, "vector": embedding, "payload": {"text": text}}]
    )

In [7]:
def retrieve_relevant_docs(question, collection_name):
    query_embedding = embeddings_model.embed_query(question)
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=5
    )
    return [hit.payload["text"] for hit in search_result]

In [8]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

In [9]:
def generate_response(question, prompt_template, collection_name):
    context_docs = retrieve_relevant_docs(question, collection_name)
    context_text = "\n\n".join(context_docs)
    print(f"\n{context_text}\n")

    messages = [
        HumanMessage(content=prompt_template.format(context=context_text, question=question))
    ]

    response = llm.invoke(messages)
    return response.content

In [10]:
questions = [
    "What type of data was LLaMA trained on?",
    "What activation functions are commonly used in LLMs?",
    "What are the main differences between the BERT model and the GPT model?",
    "What are the characteristics of the T5 model?",
    "What are the most significant challenges typical for LLMs?"
]


In [11]:
rag_template = """You are a helpful assistant specialized in answering questions about Large Language Models (LLMs).
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Answer:"""

prompt_template_rag = ChatPromptTemplate.from_template(rag_template)

recursive_splitter_answers = []

for question in questions:

    print("\nQuestion: ", question)
    print("Context:\n")
    answer = generate_response(question, prompt_template_rag, RECURSIVE_COLLECTION_NAME)
    recursive_splitter_answers.append(answer)
    print("Answer: ", answer)


Question:  What type of data was LLaMA trained on?
Context:


power of 800 GPUs[73]. No information regarding the
duration of GPU training is available.
MT-NLG: MT-NLG has a huge size of 530 billion
parameters. It has been trained on a massive dataset of
270 billion tokens, utilizing 4480 80GB A100 GPUs[74].
No data regarding the duration of GPU training is available.
The model integrates context learning features also.
LLaMA: LLaMA is a language model with an enormous
capacity with a total of 65 billion parameters. It has
undergone pre-training on a large dataset consisting of
1.4 trillion tokens. This training process was carried out
utilizing 2048 high-performance 80GB A100 GPUs[75]. The
training period is explicitly set to 21 days.
LLaMA 2: LLaMA 2 is equipped with a total of 70 billion
parameters and has performed pre-training on 2 trillion
tokens, utilizing 2000 80GB A100 GPUs [76]. The training
period is set to 25 days, and the model also contains context-
based learning.
Falco

In [12]:
def generate_response_llm(question, prompt_template):
    messages = [
        HumanMessage(content=prompt_template.format(question=question))
    ]

    response = llm.invoke(messages)
    return response.content

In [13]:
template_llm = """You are a helpful assistant specialized in answering questions about Large Language Models (LLMs).
Answer the following question to the best of your ability. Be precise and concise

Question: {question}

Answer:"""

prompt_template_llm = ChatPromptTemplate.from_template(template_llm)

pure_llm_answers = []

for question in questions:
    print("\nQuestion: ", question)
    answer = generate_response_llm(question, prompt_template_llm)
    pure_llm_answers.append(answer)
    print("Answer: ", answer)


Question:  What type of data was LLaMA trained on?
Answer:  Llama is a large language model developed by Meta, and it was trained on a massive dataset of text from various sources, including but not limited to:

* Web pages
* Books
* Articles
* Research papers
* User-generated content (e.g., forums, social media)

The specific details about the training data are not publicly disclosed by Meta, but it's known that Llama was trained on a large corpus of text in multiple languages.

Question:  What activation functions are commonly used in LLMs?
Answer:  Commonly used activation functions in Large Language Models (LLMs) include:

1. ReLU (Rectified Linear Unit): The most widely used activation function, which sets all negative values to zero.
2. GELU (Gaussian Error Linear Unit): A smooth and differentiable variant of ReLU, which is often used in transformer-based models.
3. Swish: A self-gated activation function that has been shown to improve performance on certain tasks.

These activa

In [14]:
semantic_splitter_answers = []
for question in questions:

    print("\nQuestion: ", question)
    print("Context:\n")
    answer = generate_response(question, prompt_template_rag, SEMANTIC_COLLECTION_NAME)
    semantic_splitter_answers.append(answer)
    print("Answer: ", answer)


Question:  What type of data was LLaMA trained on?
Context:


This training process was carried out
utilizing 2048 high-performance 80GB A100 GPUs[75]. The
training period is explicitly set to 21 days. LLaMA 2: LLaMA 2 is equipped with a total of 70 billion
parameters and has performed pre-training on 2 trillion
tokens, utilizing 2000 80GB A100 GPUs [76]. The training
period is set to 25 days, and the model also contains context-
based learning. Falcon: Falcon, equipped with 40 billion parameters,
undergoes pre-training on a large dataset of 1.3 trillion
tokens [77]. No details regarding the duration of GPU training
and it also have the context learning features. Chinchilla: Chinchilla is a language model that has
70 billion parameters and has been pre-trained on 1.4 trillion
tokens [78]. There is no details regarding the duration of GPU
training. OPT: OPT, equipped with 175 billion parameters, con-
ducts pre-training on 180 billion tokens utilizing 992 A100
GPUs with a capacity of 80

In [17]:
for i in range(len(questions)):
    print(f"Question:\n{questions[i]}")
    print(f"\nPure LLM response:\n{pure_llm_answers[i]}")
    print(f"\nRAG with recursive splitter:\n{recursive_splitter_answers[i]}")
    print(f"\nRAG with semantic splitter:\n{semantic_splitter_answers[i]}")
    print("-"*80)

Question:
What type of data was LLaMA trained on?

Pure LLM response:
Llama is a large language model developed by Meta, and it was trained on a massive dataset of text from various sources, including but not limited to:

* Web pages
* Books
* Articles
* Research papers
* User-generated content (e.g., forums, social media)

The specific details about the training data are not publicly disclosed by Meta, but it's known that Llama was trained on a large corpus of text in multiple languages.

RAG with recursive splitter:
According to the provided context, LLaMA was pre-trained on a large dataset consisting of 1.4 trillion tokens, which is categorized as "webpages" (87%), conversation data (5%), books and news (2%), scientific data (3%), and codes (5%).

RAG with semantic splitter:
The text does not explicitly state what type of data LLaMA was trained on. However, it is mentioned that the paper discusses large language models (LLMs) and their applications, but it does not provide specific 

W przeprowadzonych eksperymentach odpowiedzi tradycyjnego modelu LLM okazały się najbardziej kompleksowe, uwzględniające najszerszy zakres informacji. Odpowiedzi z wykorzystaniem RAGa ograniczyły się do informacji zawartych w dokumencie. Wnika to z faktu, że informacje, których wymagały postawione pytania mieściły się w zakresie wiedzy modelu językowego. Natomiast przewaga systemów opartch o RAG najbardziej uwidacznia się w przypadku koniecznośći uzyskania odpowiedzi na bardziej specyficzne pytania, które wykraczją poza zakres danych treningowych modelu. Wówczas dzięki zastosowaniu RAGa możliwe jest dostosowanie modelu do specyficznych zadń, poprzez dostarczenie odpowiedniego kontekstu w postaci dokumentów. 

W przeprowadzonych eksperymentach RAG został przetestowany z dwoma różnymi sposobami chunkowania – Recursive Character Splitter i Semantic Chunker. Druga metoda, bardziej eksperymentalna, wykorzystuje embeddingi do wyodrębniania fragmentów powiązanych semantycznie. W przypadku tej metody kontekst był zazwyczaj dłuższy, jednak nie zawsze uwzględniał najistotniejsze informacje z punktu widzenia odpowiedzi na pytania, co przełożyło się na gorsze rezultaty.

## Questions:

1. How does RAG improve the quality and reliability of LLM responses compared to pure LLM generation?
   
   Dzięki zastosowaniu RAGa, LLM może korzystać z dodatkowego kontekstu dostarczanego przez zewnętrzne źródła danych, co jest szczególnie przydatne, gdy model potrzebuje dostępu do informacji, które nie były dostępne podczas jego trenowania. RAG umożliwia uzyskanie specyficznych informacji na podstawie wybranych dokumentów, co prowadzi do bardziej precyzyjnych odpowiedzi. Odpowiedzi te nie są ograniczone do wiedzy zakodowanej w modelu podczas trenowania, ale mogą być dynamicznie rozszerzane o aktualne i specjalistyczne dane. Dzięki zastosowaniu RAGa można szybko dostosować LLM do specyficznych zadań bez konieczności czasochłonnego fine-tuningu. 
2. What are the key factors affecting RAG performance (chunk size, embedding quality, prompt design)?
   
   Wszystkie wymienione elementy (rozmiar chunków, jakość embeddingów i projektowanie promptów) mają istotne znaczeine dla jakości odpowiedzi generowanych przez model, jednak wydaje się, że w przypadku RAGa proces chunkowania ma szczególne znaczenie. Zarówno rozmiar chunków jak i technika podziału mają duży wpływ na uzyskiwane rezultaty. Zbyt mały rozmiar chunków może sprawić, że kontekst stanie się zbyt krótki, co prowadzi do pominięcia istotnych informacji. Z kolei zbyt duże fragmenty mogą powodować, że model nie uchwyci kluczowych informacji, które są istotne dla generowania odpowiedzi. Ważne jest, aby zastosować odpowiedni splitter, który zapewni, że fragmenty tworzące spójną całość nie zostaną rozdzielone. Należy również pamiętać, że prosty podział dokumentu na fragmenty może nie być wystarczający. Dokumenty często zawierają różnorodne dane, takie jak tabele, wykresy czy obrazy, które są istotne dla treści, ale muszą być najpierw odpowiednio sparsowane, aby mogły być przetworzone przez model. Odpowiednie przygotowanie tych elementów jest kluczowe dla uzyskania wysokiej jakości odpowiedzi w systemie RAG.
3. How does the choice of vector database and embedding model impact system performance?
   
   Jakości modelu embeddingowego wpływa na dokładność odwzorowywania semantycznego znaczenia tekstu. Lepsze embeddingi prowadzą do bardziej trafnych wyników wyszukiwania, co bezpośrednio wpływa na jakość odpowiedzi.

   Efektywność wektorowej bazy danych w wyszukiwaniu najbliższych sąsiadów wpływa na czas odpowiedzi systemu. Bazy zoptymalizowane pod kątem szybkiego indeksowania i wyszukiwania zapewniają szybszy dostęp do odpowiednich dokumentów. Bazy danych, które umożliwiają filtrowanie wyników na podstawie metadanych, np. typu dokumentu, zwiększają trafność wyszukiwań. 
4. What are the main challenges in implementing a production-ready RAG system?

   Wśród wyzwań związanych z implementacją systemów RAG w środowisku produkcyjnym można wyróznić długi czas przetwarzania. Zapytania wykonywane z wykorzystaniem RAGa miały kilkukrotnie dłuższy czas przetwrzania w porównianiu z tradycyjnym modelem LLM. Kolejnym istotnym wyzwaniem jest zależność jakości otrzymywanych wyników od chunkowania. Wiedza modelu w przypadku wykorzystania RAGa ogranicza się do informacji zawartych w chunkach, przez co model ma trudności w odpwiadaniu na pytania dotyczące szerszego kontekstu, np. wniosków płynących z całego dokumentu. Istotnym wyzwaniem jest także przetwarzanie różnorodnych foramtów danych, które często wymagją zaawansowanych technik parsowania, gdyż modele nie są w stanie ich prprawnie przetwarzać. 
5. How can the system be improved to handle complex queries requiring multiple document lookups?

   System RAG można usprawnić, aby lepiej radził sobie ze złożonymi zapytaniami wymagającymi wyszukiwania w wielu dokumentach poprzez zastosowanie wyszukiwania iteracyjnego (Multi-Hop Retrieval), które polega na wieloetapowym procesie przeszukiwania. W tej metodzie system najpierw przeprowadza początkowe wyszukiwanie, aby znaleźć podstawowe informacje związane z zapytaniem. Następnie, na podstawie wyników tego pierwszego wyszukiwania, formułowane są kolejne, bardziej szczegółowe podzapytania, które prowadzą do dalszych etapów przeszukiwania dokumentów.
   